In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
''' ////////////////////////////////////////////////////////////////////////////
# AUTOR: ARI JEFFERSON CARDOZO DO SACRAMENTO
# DATA: 29/07/2024
# FINALIDADE: Script para obtenção de dados das Pesquisas de Producao Fisica Industrial do IBGE - PIM-PF
# DADOS:
  1 - PIM-PF - Tabela 8888 para Producao Industrial Geral / Maquinas e Equipamentos / Industria Farmaceutica
  2 - PIM-PF - Tabela
//////////////////////////////////////////////////////////////////////////// '''
#### Bibliotecas
# Certificar das instalacaoes antes de executar
## pip install requests pandas openpyxl
import pandas as pd
import numpy as np
import requests
import time
import csv

#### Para obtenção da URL correta utilizar o Query builder do IBGE
# https://servicodados.ibge.gov.br/api/docs/agregados?versao=3#api-bq

##########################################################################################################################################################
####################################################################### APIs URLs ########################################################################
##########################################################################################################################################################

############################################################### INDICES GERAIS POR MES/ANO ###############################################################
##############################
### PIM - Producao Industrial
##############################
# Tabela 8888 | Variavel 11601 | CNAE 544[129314] - Indice de Producao Industrial Mes x Mes Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_pi_mes_mes_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11601?localidades=N1[all]&classificacao=544[129314]"
# Tabela 8888 | Variavel 11602 | CNAE 544[129314] - Indice de Producao Industrial Mes x Mesmo Mes do ano Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_pi_mes_mes_ano_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11602?localidades=N1[all]&classificacao=544[129314]"
# Tabela 8888 | Variavel 11604 | CNAE 544[129314] - Indice de Producao Industrial Acumulada em 12 meses - Relacao Periodo Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_pi_acumulado12_periodo_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11604?localidades=N1[all]&classificacao=544[129314]"

##############################
### PIM Maquinas e Equipamentos
##############################
# Tabela 8888 | Variavel 11601 | CNAE 544[129337] - Indice de Producao de Maquinas e Equipamentos Mes x Mes Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_me_mes_mes_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11601?localidades=N1[all]&classificacao=544[129337]"
# Tabela 8888 | Variavel 11602 | CNAE 544[129337] - Indice de Producao de Maquinas e Equipamentos Mes x Mesmo Mes do ano Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_me_mes_mes_ano_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11602?localidades=N1[all]&classificacao=544[129337]"
# Tabela 8888 | Variavel 11604 | CNAE 544[129337] - Indice de Producao de Maquinas e Equipamentos Acumulada em 12 meses - Relacao Periodo Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_me_acumulado12_periodo_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11604?localidades=N1[all]&classificacao=544[129337]"

##############################
### PIM Industria Farmaceutica
##############################
# Tabela 8888 | Variavel 11601 | CNAE 544[129330] - Indice de Producao da Industrial Farmaceutica Mes x Mes Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_farm_mes_mes_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11601?localidades=N1[all]&classificacao=544[129330]"
# Tabela 8888 | Variavel 11602 | CNAE 544[129330] - Indice de Producao da Industrial Farmaceutica Mes x Mesmo Mes do ano Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_farm_mes_mes_ano_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202205|202206|202207|202208|202209|202210|202211|202212|202301|202302|202303|202304|202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11602?localidades=N1[all]&classificacao=544[129330]"
# Tabela 8888 | Variavel 11604 | CNAE 544[129330] - Indice de Producao Industrial Farmaceutica Acumulada em 12 meses - Relacao Periodo Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_farm_acumulado12_periodo_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11604?localidades=N1[all]&classificacao=544[129330]"

##############################
### PIM Industria de Transformacao
##############################
# Tabela 8888 | Variavel 11601 | CNAE 544[129316] - Indice de Producao da Industria de Transformacao Mes x Mes Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_trans_mes_mes_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11601?localidades=N1[all]&classificacao=544[129316]"
# Tabela 8888 | Variavel 11602 | CNAE 544[129316] - Indice de Producao da Industria de Transformacao Mes x Mesmo Mes do ano Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_trans_mes_mes_ano_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11602?localidades=N1[all]&classificacao=544[129316]"
# Tabela 8888 | Variavel 11604 | CNAE 544[129316] - Indice de Producao da Industria de Transformacao Acumulada em 12 meses - Relacao Periodo Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_trans_acumulado12_periodo_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11604?localidades=N1[all]&classificacao=544[129316]"

##############################
### PIM Materiais Medicos/Odontologicos e Opticos
##############################
# Tabela 8885 | Variavel 11601 | CNAE 544[129276] - Indice de Producao da Industrial Materiais Medicos/Odontologicos e Opticos Mes x Mes Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
## Não há o indicador de materiais mes - Atribuindo Dataframe zerado em mesmo padrao aos demais
url_pim_mat_mes_mes_anterior = pd.DataFrame(0, index=range(12), columns=['Periodo', 'Indice'])
# Tabela 8885 | Variavel 11602 | CNAE 544[129276] - Indice de Producao da Industrial Materiais Medicos/Odontologicos e Opticos Mes x Mesmo Mes do ano Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_mat_mes_mes_ano_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8885/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11602?localidades=N1[all]&classificacao=542[129276]"
# Tabela 8885 | Variavel 11604 | CNAE 544[129276] - Indice de Producao da Industria de Transformacao Acumulada em 12 meses - Relacao Periodo Anterior - Considera os ultimos dois anos a partir do mes atual vigente (mediante disponibilidade IBGE)
url_pim_mat_acumulado12_periodo_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8885/periodos/202305|202306|202307|202308|202309|202310|202311|202312|202401|202402|202403|202404|202405/variaveis/11604?localidades=N1[all]&classificacao=542[129276]"


################################################################# INDICES COMPLETOS DE TODOS OS CNAES DO MES VIGENTE DA EXECUCAO ###################################################################
##############################
### PIM-PF TABELA 8888
##############################
# Variação (%) mês/mesmo mes do ano anterior (M/M-12) da producao fisica industrial
url_tabela8888_mes_mesmo_mes_ano_anterior = "https://servicodados.ibge.gov.br/api/v3/agregados/8888/periodos/202405/variaveis/11602?localidades=N1[all]&classificacao=544[all]"


##########################################################################################################################################################
######################################################################### Requests #######################################################################
##########################################################################################################################################################

############################################################### INDICES GERAIS POR MES/ANO ###############################################################

##################################################################
# Requisicao para Indice de Producao Industrial Mes x Mes Anterior
##################################################################
# Requisicao
response_pim_pi_mes_mes_anterior = requests.get(url_pim_pi_mes_mes_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_pi_mes_mes_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_pi_mes_mes_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_pi_mes_mes_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial - Mes x Mes Ano Vigente - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial - Mes x Mes Ano Vigente - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
############################################################################
# Requisicao para Indice de Producao Industrial Mes x Mesmo Mes Ano Anterior
############################################################################
# Requisicao
response_pim_pi_mes_mes_ano_anterior = requests.get(url_pim_pi_mes_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_pi_mes_mes_ano_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_pi_mes_mes_ano_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_pi_mes_mes_ano_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial - Mes x Mesmo Mes Ano Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial - Mes x Mesmo Mes Ano Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
############################################################################
# Requisicao para Indice de Producao Industrial Acumulada em 12 meses - Relacao Periodo Anterior
############################################################################
# Requisicao
response_pim_pi_acumulado12_periodo_anterior = requests.get(url_pim_pi_acumulado12_periodo_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_pi_acumulado12_periodo_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_pi_acumulado12_periodo_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_pi_acumulado12_periodo_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial - Acumulada em 12 meses Relacao Periodo Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial - Acumulada em 12 meses Relacao Periodo Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
#############################################################################################
# ///////////////////////////////////////////////////////////////////////////////////////// #
#############################################################################################
#############################################################################################
# Requisicao para Indice de Producao Industrial de Maquinas e Equipamentos Mes x Mes Anterior
#############################################################################################
# Requisicao
response_pim_me_mes_mes_anterior = requests.get(url_pim_me_mes_mes_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_me_mes_mes_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_me_mes_mes_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_me_mes_mes_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial de Maquinas e Equipamentos - Mes x Mes Ano Vigente - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial de Maquinas e Equipamentos - Mes x Mes Ano Vigente - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
########################################################################################################
# Requisicao para Indice de Producao Industrial de Maquinas e Equipamentos Mes x Mesmo Mes Ano Anterior
########################################################################################################
# Requisicao
response_pim_me_mes_mes_ano_anterior = requests.get(url_pim_me_mes_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_me_mes_mes_ano_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_me_mes_mes_ano_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_me_mes_mes_ano_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial de Maquinas e Equipamentos - Mes x Mesmo Mes Ano Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial de Maquinas e Equipamentos - Mes x Mesmo Mes Ano Anterior - Sucesso")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
############################################################################
# Requisicao para Indice de Producao Industrial de Maquinas e Equipamentos Acumulado em 12 meses - Relacao Periodo Anterior
############################################################################
# Requisicao
response_pim_me_acumulado12_periodo_anterior = requests.get(url_pim_me_acumulado12_periodo_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_me_acumulado12_periodo_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_me_acumulado12_periodo_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_me_acumulado12_periodo_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial de Maquinas e Equipamentos - Acumulada em 12 meses Relacao Periodo Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial de Maquinas e Equipamentos - Acumulada em 12 meses Relacao Periodo Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
#############################################################################################
# ///////////////////////////////////////////////////////////////////////////////////////// #
#############################################################################################
#############################################################################################
# Requisicao para Indice de Producao Industrial da Industria Farmaceutica Mes x Mes Anterior
#############################################################################################
# Requisicao
response_pim_farm_mes_mes_anterior = requests.get(url_pim_farm_mes_mes_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_farm_mes_mes_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_farm_mes_mes_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_farm_mes_mes_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial da Industria Farmaceutica - Mes x Mes Ano Vigente - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial da Industria Farmaceutica - Mes x Mes Ano Vigente - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
########################################################################################################
# Requisicao para Indice de Producao da Industria Farmaceutica Mes x Mesmo Mes Ano Anterior
########################################################################################################
# Requisicao
response_pim_farm_mes_mes_ano_anterior = requests.get(url_pim_farm_mes_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_farm_mes_mes_ano_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_farm_mes_mes_ano_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_farm_mes_mes_ano_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial da Industria Farmaceutica - Mes x Mesmo Mes Ano Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial da Industria Farmaceutica - Mes x Mesmo Mes Ano Anterior - Sucesso")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
############################################################################
# Requisicao para Indice de Producao da Industria Farmaceutica Acumulado em 12 meses - Relacao Periodo Anterior
############################################################################
# Requisicao
response_pim_farm_acumulado12_periodo_anterior = requests.get(url_pim_farm_acumulado12_periodo_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_farm_acumulado12_periodo_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_farm_acumulado12_periodo_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_farm_acumulado12_periodo_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial da Industria Farmaceutica - Acumulado em 12 meses Relacao Periodo Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial da Industria Farmaceutica - Acumulado em 12 meses Relacao Periodo Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
##############################################################################################################################
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////// #
##############################################################################################################################

#####################################################################################
# Requisicao para Indice de Producao da Industria de Transformacao Mes x Mes Anterior
#####################################################################################
# Requisicao
response_pim_trans_mes_mes_anterior = requests.get(url_pim_trans_mes_mes_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_trans_mes_mes_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_trans_mes_mes_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_trans_mes_mes_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial da Industria de Transformacao - Mes x Mes Ano Vigente - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial da Industria de Transformacao - Mes x Mes Ano Vigente - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
###############################################################################################
# Requisicao para Indice de Producao da Industria de Transformacao Mes x Mesmo Mes Ano Anterior
###############################################################################################
# Requisicao
response_pim_trans_mes_mes_ano_anterior = requests.get(url_pim_trans_mes_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_trans_mes_mes_ano_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_trans_mes_mes_ano_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_trans_mes_mes_ano_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial da Industria de Transformacao - Mes x Mesmo Mes Ano Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial da Industria de Transformacao - Mes x Mesmo Mes Ano Anterior - Sucesso")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
###################################################################################################################
# Requisicao para Indice de Producao da Industria de Transformacao Acumulado em 12 meses - Relacao Periodo Anterior
###################################################################################################################
# Requisicao
response_pim_trans_acumulado12_periodo_anterior = requests.get(url_pim_trans_acumulado12_periodo_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_trans_acumulado12_periodo_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_trans_acumulado12_periodo_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_trans_acumulado12_periodo_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial da Industria de Transformacao - Acumulado em 12 meses Relacao Periodo Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial da Industria de Transformacao - Acumulado em 12 meses Relacao Periodo Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
##############################################################################################################################
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////// #
##############################################################################################################################

##############################################################################################################################
# Requisicao para Indice de Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos Mes x Mes Anterior
##############################################################################################################################
# Requisicao (Tentando pela proxima disponivel da mesma tabela pois a primeira nao existe no IBGE - Unico Caso)
response_pim_mat_mes_mes_anterior = requests.get(url_pim_mat_mes_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_mat_mes_mes_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_mat_mes_mes_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': 0})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_mat_mes_mes_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos Mes x Mes Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos Mes x Mes Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
########################################################################################################################################
# Requisicao para Indice de Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos Mes x Mesmo Mes Ano Anterior
########################################################################################################################################
# Requisicao
response_pim_mat_mes_mes_ano_anterior = requests.get(url_pim_mat_mes_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_mat_mes_mes_ano_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_mat_mes_mes_ano_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_mat_mes_mes_ano_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos - Mes x Mesmo Mes Ano Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos - Mes x Mesmo Mes Ano Anterior - Sucesso")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
############################################################################################################################################################
# Requisicao para Indice de Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos Acumulado em 12 meses - Relacao Periodo Anterior
############################################################################################################################################################
# Requisicao
response_pim_mat_acumulado12_periodo_anterior = requests.get(url_pim_mat_acumulado12_periodo_anterior)

# Verifica se a requisicao e bem-sucedida
if response_pim_mat_acumulado12_periodo_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_pim_mat_acumulado12_periodo_anterior.json()

    # Extraindo a serie do IBGE
    series_data = []

    # Percorrendo o JSON para extrair os dados necessarios
    for result in data[0]['resultados']:
        for serie in result['series']:
            for periodo, valor in serie['serie'].items():
                series_data.append({'Periodo': periodo, 'Indice': valor})

    # Criando um DataFrame com os dados
    df = pd.DataFrame(series_data)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_pim_mat_acumulado12_periodo_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos - Acumulado em 12 meses Relacao Periodo Anterior - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Producao Industrial de Instrumentos e Materiais Medicos/Odontologicos e Opticos - Acumulado em 12 meses Relacao Periodo Anterior - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
#############################################################################################
# ///////////////////////////////////////////////////////////////////////////////////////// #
#############################################################################################
###################################################################################################################################
# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////// #
###################################################################################################################################
###################################################################################################################################################################################################
# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////// #
###################################################################################################################################################################################################

################################################################# INDICES COMPLETOS DE TODOS OS CNAES DO MES VIGENTE DA EXECUCAO ###################################################################

####################################################################################################################################################################################################
# Tabela 8888 - Variacao (%) mes/mesmo mes do ano anterior (M/M-12) da produção fisica industrial - Todos os CNAES
####################################################################################################################################################################################################
# Requisicao
response_tabela8888_mes_mesmo_mes_ano_anterior = requests.get(url_tabela8888_mes_mesmo_mes_ano_anterior)

# Verifica se a requisicao e bem-sucedida
if response_tabela8888_mes_mesmo_mes_ano_anterior.status_code == 200:

    # Convertendo a resposta em JSON
    data = response_tabela8888_mes_mesmo_mes_ano_anterior.json()

    # Preparando para receber a serie do IBGE
    dados = []

    # Para remover numros e ponto do texto da categoria
    limpa_categoria = str.maketrans('', '', '0123456789.')

    # Percorrendo o JSON para extrair os dados necessarios
    for resultado in data:
        for res in resultado.get('resultados', []):
            classificacoes = res.get('classificacoes', [])
            for classificacao in classificacoes:
                nome_classificacao = classificacao.get('nome')
                for id_categoria, nome_categoria in classificacao.get('categoria', {}).items():
                    # Iterar sobre as séries
                    series = res.get('series', [])
                    for serie in series:
                        localidade = serie.get('localidade', {}).get('nome')
                        for periodo, valor in serie.get('serie', {}).items():
                            # Adicionar cada registro à lista de dados
                            dados.append({
                                'Periodo': periodo,
                                #'Nome Classificação': nome_classificacao,
                                'CNAE': id_categoria,
                                'Categoria': nome_categoria.translate(limpa_categoria).lstrip(), #lstrip para remover espacos em branco a esquerda que ficam apos o limpa_categoria
                                #'Localidade': localidade,
                                'Indice': valor
                            })

    # Criar um DataFrame do pandas a partir da lista de dados
    df = pd.DataFrame(dados)

    # Salvando os dados em um arquivo Excel
    df.to_excel("/content/drive/MyDrive/13 - TI/2 - Desenvolvimento/2 - Python/1_Colab Notebooks/7_MinisterioSaude/TR1/tab_tabela8888_mes_mesmo_mes_ano_anterior.xlsx", index=False)

    # Retorno de exito
    print("PIM - Tabela 8888 - Variacao (%) mes/mesmo mes do ano anterior (M/M-12) da produção fisica industrial - Todos os CNAES - Sucesso")

else:

    # Imprime uma mensagem de erro
    print("PIM - Tabela 8888 - Variacao (%) mes/mesmo mes do ano anterior (M/M-12) da produção fisica industrial - Todos os CNAES - Falha")

# Aguardar 3 segundos entre uma requisicao e outra para nao caracterizar DDoS
time.sleep(3)
#############################################################################################
# ///////////////////////////////////////////////////////////////////////////////////////// #
#############################################################################################

PIM - Producao Industrial - Mes x Mes Ano Vigente - Sucesso
PIM - Producao Industrial - Mes x Mesmo Mes Ano Anterior - Sucesso
PIM - Producao Industrial - Acumulada em 12 meses Relacao Periodo Anterior - Sucesso
PIM - Producao Industrial de Maquinas e Equipamentos - Mes x Mes Ano Vigente - Sucesso
PIM - Producao Industrial de Maquinas e Equipamentos - Mes x Mesmo Mes Ano Anterior - Sucesso
PIM - Producao Industrial de Maquinas e Equipamentos - Acumulada em 12 meses Relacao Periodo Anterior - Sucesso
PIM - Producao Industrial da Industria Farmaceutica - Mes x Mes Ano Vigente - Sucesso
PIM - Producao Industrial da Industria Farmaceutica - Mes x Mesmo Mes Ano Anterior - Sucesso
PIM - Producao Industrial da Industria Farmaceutica - Acumulado em 12 meses Relacao Periodo Anterior - Sucesso
PIM - Producao Industrial da Industria de Transformacao - Mes x Mes Ano Vigente - Sucesso
PIM - Producao Industrial da Industria de Transformacao - Mes x Mesmo Mes Ano Anterior - Sucesso
PIM - Producao Ind